# Homework 2: Data from the Web

In [1]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import time

In [2]:
URL_init = 'https://www.topuniversities.com/university-rankings/world-university-rankings/2018'

In [3]:
r_init = requests.get(URL_init)
soup = BeautifulSoup(r_init.text, 'html5lib') # This parser should be the closest one to Chorome Inspector.

In [4]:
inner_panel = soup.find('div', {'class': 'panel-panel-inner'})
print(inner_panel.prettify()[:500])

<div class="panel-panel-inner">
 <div class="panel-pane pane-block pane-block-7 pane-node-field-article-date">
  <div class="pane-content">
   <div class="pane-qs-wur-app">
    <a class="app-store-badge" href="https://itunes.apple.com/us/app/qs-world-university-rankings/id924021289?ls=1&amp;mt=8">
     <img alt="" class="media-image" height="39" src="/sites/all/themes/topuni/assets/images/svg/appstore.svg" title="" width="100"/>
    </a>
    <a class="play-badge" href="https://play.google.com/st


Looks like what we've got in Chrome Inspector!

<img src="./pics/inner_panel.png" alt="inner_panel" style="width: 600px;" />

But if we take a closer look at where might be the data we are looking for ...

<img src="./pics/thead-tbody.png" alt="thead" style="width: 600px;"/>

In [5]:
inner_panel.find('thead') is None

False

In [6]:
inner_panel.find('tbody') is None

True

... whoops! `<thead>` is there, `<tbody>`, however, is not. Must be side-loaded via JavaScript.

Let's take a look at what kind of XHRs are there ...

<img src="./pics/xhr.png" alt="tbody" style="width: 600px;"/>

Interesting `.txt` file of the largest size and loading time, let's see what's inside ...

In [7]:
URL = 'https://www.topuniversities.com/sites/default/files/qs-rankings-data/357051.txt'

In [8]:
data = requests.get(URL).json()
print(json.dumps(data, sort_keys=True, indent=4)[:500])
print('\n...\n')
print(json.dumps(data, sort_keys=True, indent=4)[-500:])

{
    "data": [
        {
            "cc": "US",
            "core_id": "410",
            "country": "United States",
            "guide": "<a href=\"/where-to-study/north-america/united-states/guide\" class=\"guide-link\" target=\"_blank\">United States</a>",
            "logo": "<img src=\"https://www.topuniversities.com/sites/default/files/massachusetts-institute-of-technology-mit_410_small_0.jpg\" alt=\"Massachusetts Institute of Technology (MIT)  Logo\">",
            "nid": "294850",
   

...

a>",
            "logo": "<img src=\"https://www.topuniversities.com/sites/default/files/wroclaw-university-of-technology_592560cf2aeae70239af5039_small.jpg\" alt=\"Wroclaw University of Technology Logo\">",
            "nid": "296503",
            "rank_display": "801-1000",
            "region": "Europe",
            "score": "",
            "stars": null,
            "title": "Wroclaw University of Technology",
            "url": "/universities/wroclaw-university-technology"
        }


That's exactly the data we were looking for, no need to web-scrape anymore, yay!

In [9]:
df = pd.DataFrame(data['data'])
df.head()

cc core_id         country  \
0  US     410   United States   
1  US     573   United States   
2  US     253   United States   
3  US      94   United States   
4  GB      95  United Kingdom   

                                               guide  \
0  <a href="/where-to-study/north-america/united-...   
1  <a href="/where-to-study/north-america/united-...   
2  <a href="/where-to-study/north-america/united-...   
3  <a href="/where-to-study/north-america/united-...   
4  <a href="/where-to-study/europe/united-kingdom...   

                                                logo     nid rank_display  \
0  <img src="https://www.topuniversities.com/site...  294850            1   
1  <img src="https://www.topuniversities.com/site...  297282            2   
2  <img src="https://www.topuniversities.com/site...  294270            3   
3  <img src="https://www.topuniversities.com/site...  294562            4   
4  <img src="https://www.topuniversities.com/site...  294561            5   

          region score stars                                         title  \
0  North America   100     6   Massachusetts Institute of Technology (MIT)   
1  North America  98.7     5                           Stanford University   
2  North America  98.4     5                            Harvard University   
3  North America  97.7     5  California Institute of Technology (Caltech)   
4         Europe  95.6     5                       University of Cambridge   

                                                 url  
0  /universities/massachusetts-institute-technolo...  
1                  /universities/stanford-university  
2                   /universities/harvard-university  
3  /universities/california-institute-technology-...  
4                 /universities/university-cambridge

Let's drop the features that we don't need for sure.
We need these:
- *name* (**title**)
- *rank* (**rank_display**)
- *country* (**country**) and *region* (**region**)
- *number of faculty members (international and total)* and *number of students (international and total)* (might need **url** to extract these).

In [10]:
pretty_df = df[['title', 'rank_display', 'stars', 'country', 'region', 'url']]

In [55]:
pretty_df.head()

title rank_display stars  \
0   Massachusetts Institute of Technology (MIT)            1     6   
1                           Stanford University            2     5   
2                            Harvard University            3     5   
3  California Institute of Technology (Caltech)            4     5   
4                       University of Cambridge            5     5   

          country         region  \
0   United States  North America   
1   United States  North America   
2   United States  North America   
3   United States  North America   
4  United Kingdom         Europe   

                                                 url  
0  /universities/massachusetts-institute-technolo...  
1                  /universities/stanford-university  
2                   /universities/harvard-university  
3  /universities/california-institute-technology-...  
4                 /universities/university-cambridge

Now let's for each uni follow its link in and scrape the numbers that we need:
- *number of faculty members (international and total)*
- *number of students (international and total)*

In [47]:
def get_number(tot):
    """Gets the number of people."""
    str_ = tot.find('div', class_='number').text
    return int(''.join(filter(str.isdigit, str_)))

In [48]:
def get_totals(daf):
    total_student = []
    total_student_inter = []
    total_faculty = []
    total_faculty_inter = []
    
    for url in daf.url:
        URL = 'https://www.topuniversities.com' + url
        uni = BeautifulSoup(requests.get(URL).text, 'html5lib')
        
        tot = uni.find('div', class_='total student')
        if tot is None:
            total_student.append(-1)
        else:
            total_student.append(get_number(tot))
         
        tot = uni.find('div', class_='total inter')
        if tot is None:
            total_student_inter.append(-1)
        else:
            total_student_inter.append(get_number(tot))
        
        tot = uni.find('div', class_='total faculty')
        if tot is None:
            total_faculty.append(-1)
        else:
            total_faculty.append(get_number(tot))
            
        tot = uni.find('div', class_='inter faculty')
        if tot is None:
            total_faculty_inter.append(-1)
        else:
            total_faculty_inter.append(get_number(tot))
                
    return total_student, total_student_inter, total_faculty, total_faculty_inter

In [50]:
%time total_student, total_student_inter, total_faculty, total_faculty_inter = get_totals(pretty_df[:200])

CPU times: user 1min 39s, sys: 565 ms, total: 1min 39s
Wall time: 2min 43s


In [52]:
pretty_df_200 = pretty_df[:200]
pretty_df_200 = pretty_df_200.assign(total_student=pd.Series(total_student).values)
pretty_df_200 = pretty_df_200.assign(total_student_inter=pd.Series(total_student_inter).values)
pretty_df_200 = pretty_df_200.assign(total_faculty=pd.Series(total_faculty).values)
pretty_df_200 = pretty_df_200.assign(total_faculty_inter=pd.Series(total_faculty_inter).values)

In [54]:
pretty_df_200.head()

title rank_display stars  \
0   Massachusetts Institute of Technology (MIT)            1     6   
1                           Stanford University            2     5   
2                            Harvard University            3     5   
3  California Institute of Technology (Caltech)            4     5   
4                       University of Cambridge            5     5   

          country         region  \
0   United States  North America   
1   United States  North America   
2   United States  North America   
3   United States  North America   
4  United Kingdom         Europe   

                                                 url  total_student  \
0  /universities/massachusetts-institute-technolo...          11067   
1                  /universities/stanford-university          15878   
2                   /universities/harvard-university          22429   
3  /universities/california-institute-technology-...           2255   
4                 /universities/university-cambridge          18770   

   total_student_inter  total_faculty  total_faculty_inter  
0                 3717           2982                 1679  
1                 3611           4285                 2042  
2                 5266           4350                 1311  
3                  647            953                  350  
4                 6699           5490                 2278

That's it for today :)

*18.10.2017*